In [129]:
# Imports
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [130]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
applicant_data_df = pd.read_csv(Path('Resources/applicants_data.csv'))

# Review the DataFrame
applicant_data_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [131]:
# Review the data types associated with the columns
applicant_data_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [132]:
# Drop the 'EIN' and 'NAME' columns from the DataFrame
applicant_data_df = applicant_data_df.drop(columns=['NAME', 'EIN'])

# Review the DataFrame
applicant_data_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [133]:
# Create a list of categorical variables 
categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes == "object"].index)

# Display the categorical variables list
display(categorical_variables)

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [134]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)


In [135]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])


In [136]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(encoded_data, columns=enc.get_feature_names(categorical_variables))

# Review the DataFrame
encoded_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [137]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
encoded_df = pd.concat([encoded_df, applicant_data_df[['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL']]], axis=1)

# Review the Dataframe
encoded_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT,IS_SUCCESSFUL
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,108590,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,6692,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,142590,1


In [138]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df['IS_SUCCESSFUL']

# Display a sample of y
y[:5]

0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [139]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = encoded_df.drop(columns=['IS_SUCCESSFUL'])

# Review the features DataFrame
X.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,108590
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,6692
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,142590


In [140]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [141]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [142]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features


116

In [143]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [144]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  (number_input_features + number_output_neurons) // 2 

# Review the number hidden nodes in the first layer
hidden_nodes_layer1


58

In [145]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  (hidden_nodes_layer1 + number_output_neurons) // 2

# Review the number hidden nodes in the second layer
hidden_nodes_layer2


29

In [146]:
# Create the Sequential model instance
nn = Sequential()


In [147]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

In [148]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

In [149]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))

In [150]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 58)                6786      
_________________________________________________________________
dense_19 (Dense)             (None, 29)                1711      
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 30        
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________


In [151]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [152]:
# Fit the model using 50 epochs and the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 784us/step - loss: 0.5797 - accuracy: 0.7170
Epoch 2/50
804/804 [==============================] - 1s 807us/step - loss: 0.5543 - accuracy: 0.7283
Epoch 3/50
804/804 [==============================] - 1s 776us/step - loss: 0.5508 - accuracy: 0.7314
Epoch 4/50
804/804 [==============================] - 1s 804us/step - loss: 0.5483 - accuracy: 0.7305
Epoch 5/50
804/804 [==============================] - 1s 791us/step - loss: 0.5468 - accuracy: 0.7322
Epoch 6/50
804/804 [==============================] - 1s 773us/step - loss: 0.5457 - accuracy: 0.7327
Epoch 7/50
804/804 [==============================] - 1s 792us/step - loss: 0.5451 - accuracy: 0.7339
Epoch 8/50
804/804 [==============================] - 1s 771us/step - loss: 0.5444 - accuracy: 0.7348
Epoch 9/50
804/804 [==============================] - 1s 781us/step - loss: 0.5431 - accuracy: 0.7327
Epoch 10/50
804/804 [==============================] - 1s 773us/step - loss: 0.543

In [153]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5565 - accuracy: 0.7299
Loss: 0.5564936399459839, Accuracy: 0.729912519454956


In [154]:
# Set the model's file path
file_path = Path("Resources/AlphabetSoup.h5")

# Export your model to a HDF5 file
nn.save(file_path)

---

## Optimize the neural network model


### Alternative Model 1
#### For my first alternative NN model I removed one hidden layer

In [155]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [156]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 1

In [157]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 = (number_input_features + number_output_neurons_A1) // 2 

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A1

58

In [158]:
# Create the Sequential model instance
nn_A1 = Sequential()

In [159]:
# First hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer1_A1, input_dim=number_input_features, activation="relu"))

# Output layer
nn_A1.add(Dense(units=number_output_neurons_A1, activation="sigmoid"))

# Check the structure of the model
nn_A1.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 58)                6786      
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 59        
Total params: 6,845
Trainable params: 6,845
Non-trainable params: 0
_________________________________________________________________


In [160]:
# Compile the Sequential model
nn_A1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [161]:
# Fit the model using 50 epochs and the training data
fit_model_A1 = nn_A1.fit(X_train_scaled, y_train, epochs=50)


Epoch 1/50
804/804 [==============================] - 1s 930us/step - loss: 0.5824 - accuracy: 0.7163
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5576 - accuracy: 0.7283
Epoch 3/50
804/804 [==============================] - 1s 879us/step - loss: 0.5544 - accuracy: 0.7313
Epoch 4/50
804/804 [==============================] - 1s 809us/step - loss: 0.5517 - accuracy: 0.7310
Epoch 5/50
804/804 [==============================] - 1s 721us/step - loss: 0.5506 - accuracy: 0.7320
Epoch 6/50
804/804 [==============================] - 1s 730us/step - loss: 0.5494 - accuracy: 0.7324
Epoch 7/50
804/804 [==============================] - 1s 730us/step - loss: 0.5482 - accuracy: 0.7319
Epoch 8/50
804/804 [==============================] - 1s 722us/step - loss: 0.5476 - accuracy: 0.7321
Epoch 9/50
804/804 [==============================] - 1s 729us/step - loss: 0.5467 - accuracy: 0.7321
Epoch 10/50
804/804 [==============================] - 1s 724us/step - loss: 0.5470 

#### Alternative Model 2
#### For my second alternative NN model I doubled the number of Epochs

In [162]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [163]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 1

In [164]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 = (number_input_features + number_output_neurons_A2) // 2 

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A2

58

In [165]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A2 =  (hidden_nodes_layer1_A2 + number_output_neurons_A2) // 2

# Review the number hidden nodes in the second layer
hidden_nodes_layer2_A2


29

In [166]:
# Create the Sequential model instance
nn_A2 = Sequential()

In [167]:
# First hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer1_A2, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer2_A2, activation="relu"))

# Output layer
nn_A2.add(Dense(units=number_output_neurons_A2, activation="sigmoid"))

# Check the structure of the model
nn_A2.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 58)                6786      
_________________________________________________________________
dense_24 (Dense)             (None, 29)                1711      
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 30        
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________


In [168]:
# Compile the model
nn_A2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [169]:
# Fit the model
fit_model = nn_A2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 968us/step - loss: 0.5783 - accuracy: 0.7166
Epoch 2/100
804/804 [==============================] - 1s 927us/step - loss: 0.5538 - accuracy: 0.7305
Epoch 3/100
804/804 [==============================] - 1s 841us/step - loss: 0.5501 - accuracy: 0.7302
Epoch 4/100
804/804 [==============================] - 1s 849us/step - loss: 0.5480 - accuracy: 0.7313
Epoch 5/100
804/804 [==============================] - 1s 813us/step - loss: 0.5470 - accuracy: 0.7320
Epoch 6/100
804/804 [==============================] - 1s 867us/step - loss: 0.5461 - accuracy: 0.7334
Epoch 7/100
804/804 [==============================] - 1s 816us/step - loss: 0.5454 - accuracy: 0.7333
Epoch 8/100
804/804 [==============================] - 1s 864us/step - loss: 0.5440 - accuracy: 0.7335
Epoch 9/100
804/804 [==============================] - 1s 922us/step - loss: 0.5437 - accuracy: 0.7333
Epoch 10/100
804/804 [==============================] - 1s 865us/step - l

804/804 [==============================] - 1s 770us/step - loss: 0.5297 - accuracy: 0.7400
Epoch 80/100
804/804 [==============================] - 1s 809us/step - loss: 0.5299 - accuracy: 0.7415
Epoch 81/100
804/804 [==============================] - 1s 848us/step - loss: 0.5300 - accuracy: 0.7417
Epoch 82/100
804/804 [==============================] - 1s 768us/step - loss: 0.5302 - accuracy: 0.7422
Epoch 83/100
804/804 [==============================] - 1s 771us/step - loss: 0.5294 - accuracy: 0.7430
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5295 - accuracy: 0.7429
Epoch 85/100
804/804 [==============================] - 1s 830us/step - loss: 0.5290 - accuracy: 0.7410
Epoch 86/100
804/804 [==============================] - 1s 937us/step - loss: 0.5294 - accuracy: 0.7419
Epoch 87/100
804/804 [==============================] - 1s 937us/step - loss: 0.5292 - accuracy: 0.7425
Epoch 88/100
804/804 [==============================] - 1s 872us/step - loss: 0

#### Alternative Model 3
#### For my third alternative NN model I doubled the number of Epochs and added a hidden layer

In [170]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Define the number of neurons in the output layer
number_output_neurons_A3 = 1


# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A3 = (number_input_features + number_output_neurons_A3) // 2 

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A3 =  (hidden_nodes_layer1_A3 + number_output_neurons_A3) // 2

# Define the number of hidden nodes for third hidden layer
hidden_nodes_layer3_A3 =  (hidden_nodes_layer2_A3 + number_output_neurons_A3) // 2

# Create the Sequential model instance
nn_A3 = Sequential()

# First hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer1_A3, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer2_A3, activation="relu"))

# Third hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer3_A3, activation="relu"))

# Output layer
nn_A3.add(Dense(units=number_output_neurons_A3, activation="sigmoid"))

# Check the structure of the model
nn_A3.summary()


Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 58)                6786      
_________________________________________________________________
dense_27 (Dense)             (None, 29)                1711      
_________________________________________________________________
dense_28 (Dense)             (None, 15)                450       
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 16        
Total params: 8,963
Trainable params: 8,963
Non-trainable params: 0
_________________________________________________________________


In [171]:
# Compile the model
nn_A3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Fit the model
fit_model = nn_A3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 788us/step - loss: 0.5748 - accuracy: 0.7172
Epoch 2/100
804/804 [==============================] - 1s 930us/step - loss: 0.5524 - accuracy: 0.7293
Epoch 3/100
804/804 [==============================] - 1s 943us/step - loss: 0.5487 - accuracy: 0.7324
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5466 - accuracy: 0.7321
Epoch 5/100
804/804 [==============================] - 1s 971us/step - loss: 0.5451 - accuracy: 0.7336
Epoch 6/100
804/804 [==============================] - 1s 849us/step - loss: 0.5439 - accuracy: 0.7338
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5430 - accuracy: 0.7364
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5420 - accuracy: 0.7353
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5414 - accuracy: 0.7359
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.540

804/804 [==============================] - 1s 797us/step - loss: 0.5288 - accuracy: 0.7428
Epoch 81/100
804/804 [==============================] - 1s 786us/step - loss: 0.5290 - accuracy: 0.7428
Epoch 82/100
804/804 [==============================] - 1s 793us/step - loss: 0.5288 - accuracy: 0.7423
Epoch 83/100
804/804 [==============================] - 1s 802us/step - loss: 0.5284 - accuracy: 0.7429
Epoch 84/100
804/804 [==============================] - 1s 798us/step - loss: 0.5292 - accuracy: 0.7423
Epoch 85/100
804/804 [==============================] - 1s 791us/step - loss: 0.5285 - accuracy: 0.7422
Epoch 86/100
804/804 [==============================] - 1s 795us/step - loss: 0.5286 - accuracy: 0.7419
Epoch 87/100
804/804 [==============================] - 1s 796us/step - loss: 0.5287 - accuracy: 0.7423
Epoch 88/100
804/804 [==============================] - 1s 797us/step - loss: 0.5285 - accuracy: 0.7425
Epoch 89/100
804/804 [==============================] - 1s 801us/step - loss:

In [172]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Original Model Results
268/268 - 0s - loss: 0.5565 - accuracy: 0.7299
Loss: 0.5564936399459839, Accuracy: 0.729912519454956


In [173]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 1 Results
268/268 - 0s - loss: 0.5639 - accuracy: 0.7300
Loss: 0.5639050006866455, Accuracy: 0.7300291657447815


In [174]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A2.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 2 Results
268/268 - 0s - loss: 0.5619 - accuracy: 0.7298
Loss: 0.5619392991065979, Accuracy: 0.7297959327697754


In [175]:
print("Alternative Model 3 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A3.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 3 Results
268/268 - 0s - loss: 0.5611 - accuracy: 0.7294
Loss: 0.5610897541046143, Accuracy: 0.7294460535049438


In [176]:
# Set the file path for the first alternative model
file_path = Path("Resources/AlphabetSoup_A1.h5")

# Export your model to a HDF5 file
nn.save(file_path)

In [177]:
# Set the file path for the second alternative model
file_path = Path("Resources/AlphabetSoup_A2.h5")

# Export your model to a HDF5 file
nn.save(file_path)

In [178]:
# Set the file path for the third alternative model
file_path = Path("Resources/AlphabetSoup_A3.h5")

# Export your model to a HDF5 file
nn.save(file_path)